In [1]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
model = ChatOpenAI(model="gpt-4o", temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpul and kind chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
output_parser = StrOutputParser()
memory = ConversationBufferMemory(return_messages=True)
memory.load_memory_variables({})

{'history': []}

In [3]:
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"))
    | prompt
    | model
    | output_parser
)
inputs = {"input": "hi im kade"}
response = chain.invoke(inputs)
response

"Hi Kade! It's great to meet you. How can I assist you today?"

In [4]:
memory.save_context(inputs=inputs, outputs={"output": response})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi im kade'),
  AIMessage(content="Hi Kade! It's great to meet you. How can I assist you today?")]}

In [5]:
inputs = {"input": "whats my name"}
chain.invoke(inputs)

'Your name is Kade! How can I help you today?'